# Lesson 05E - Two convolutions


[Model depth](file:///C:/Users/ishay/Documents/Data%20Science/Coursera%20and%20practice/ML_Mastery/NLP/Model%20Improvement%20and%20Best%20Practices/Deep%20Learning%20for%20NLP%20Best%20Practices.html) - In most cases, however, performance improvements of making the model deeper than 2 layers are minimal (Reimers & Gurevych, 2017). These observations hold for most sequence tagging and structured prediction problems. For classification, deep or very deep models perform well only with character-level input (Chinese?) and shallow word-level models are still the state-of-the-art.

In [1]:
from keras.datasets import imdb
import pandas as pd
import numpy as np

from keras.preprocessing import sequence

Using TensorFlow backend.


**Prepare the data**

In [2]:
max_features = 5000

In [3]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features) # instead on nb_words=

I can use another argument here, which is 'skip_top=', and that to ignore X top most frequent words

In [4]:
maxlen = 400

We define maximum length of 400. It means that posts shoeter than 400 will be filled with 0s, and longer posts are cut.

In [5]:
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

The first two arrays were padded, and the third one was cut.

**Now defining the model**

As opposed to the original file, I an creating the model in a function.

Also gathering all the imports from the original version (05) in the following cell

In [36]:
from keras.models import Sequential, Model
from keras.layers import Embedding
from keras.layers import SpatialDropout1D
from keras.layers import Convolution1D
from keras.layers import Conv1D
from keras.layers import GlobalMaxPooling1D, MaxPooling1D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Input

Keras models can be used in scikit-learn by wrapping them with the **KerasClassifier** or **KerasRegressor** class.

To use these wrappers you must define a function that creates and returns your Keras sequential model, then pass this function to the build_fn argument when constructing the KerasClassifier class.

In [32]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [41]:
def create_model():
    # create model
    inputs_shape1 = Input(shape=(400,))
    embed1 = Embedding(input_dim=5000, output_dim=50)(inputs_shape1)
    spat_drop1 = SpatialDropout1D(0.4)(embed1)
    # First conv
    conv1 = Conv1D(filters=250, kernel_size=3, padding='valid', activation='relu', strides=1)(spat_drop1)
    pool1 = MaxPooling1D(pool_size=2)(conv1)
    # Second conv
    conv2 = Conv1D(filters=100, kernel_size=4, padding='valid', activation='relu', strides=2)(pool1)
    pool2 = MaxPooling1D(pool_size=2)(conv2)
    # Flatten
    flat = Flatten()(pool2)
    # Now the fully connected layers
    dense1 = Dense(250, activation='relu')(flat)
    drop1 = Dropout(0.3)(dense1)
    dense2 = Dense(20, activation='relu')(drop1)
    drop2 = Dropout(0.3)(dense2)
    output = Dense(1, activation='sigmoid')(drop2)
    model = Model(inputs=inputs_shape1, outputs=output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model
    
    return model

Within the KerasClassifier I include the original fit agruments (batch_size, epochs, verbose), unless I want to grid search them, then I include them in the param_grid

In [42]:
model = KerasClassifier(build_fn=create_model, verbose=1, epochs=1, batch_size=32)

In [43]:
epochs = 5
batch_size = 32
h = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, 
              validation_data=(X_test, y_test), verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 400)               0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 400, 50)           250000    
_________________________________________________________________
spatial_dropout1d_8 (Spatial (None, 400, 50)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 398, 250)          37750     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 199, 250)          0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 98, 100)           100100    
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 49, 100)           0         
__________

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Validation: accuracy1 = %f  ;  loss1 = %f" % (accuracy, loss))